In [ ]:
# An overall idea about the dataset to start our modeling exercise :)

# Some more understanding about the dataset like:

# Different columns and their data types
# There are no missing values
# Different categories present in the categorical variables and their distribution.
# Target variable distribution of different binary variables
# ID variable does not seem to be leaky
# Important variables from a non-linear basic model
# A lot of these ideas/code has been taken from Kaggle user: SRK (https://www.kaggle.com/sudalairajkumar)

In [1]:
##Import necessary modules

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [ ]:
## List files present in the input

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
## starting with train file exploration

In [ ]:
train_file = '' # csv file location
date_fields = []
train_df = pd.read_csv(train_file, parse_dates=date_fields)
train_df.shape

In [ ]:
train_df.head()

In [ ]:
# Analyse the label/target field

In [ ]:
label_name = ''
plt.figure(figsize=(8,6))
plt.scatter(range(train_df.shape[0]), np.sort(train_df[label_name].values))
plt.xlabel('index', fontsize=12)
plt.ylabel(label_name, fontsize=12)
plt.show()

In [ ]:
# remove outliers
# ulimit = np.percentile(train_df[label_name].values, 99)
# llimit = np.percentile(train_df[label_name].values, 1)
# train_df[label_name].ix[train_df[label_name]>ulimit] = ulimit
# train_df[label_name].ix[train_df[label_name]<llimit] = llimit


In [ ]:
# plt histogram of the label/target field

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(train_df[label_name].values, bins=50, kde=False)
plt.xlabel(label_name, fontsize=12)
plt.show()

In [ ]:
# if the target variable is skewed then we can use the log value of the target variable

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot( np.log1p(train_df[label_name].values), bins=50, kde=False)
plt.xlabel('Target', fontsize=12)
plt.title("Log of Target Histogram", fontsize=14)
plt.show()

In [ ]:
#### evaluate each of the individual fields in the dataset (beside the label)

In [ ]:
#check for missing values NAN

In [ ]:
# replace prop_df with relevant df
missing_df = prop_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.ix[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

missing_df

In [ ]:
#visualize the columns with NAN values

In [ ]:
ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

In [ ]:
# check dtype of the different variables

In [ ]:
pd.options.display.max_rows = 65

dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df

In [ ]:
## object are likely to be categorical variables

In [ ]:
# lets count the different dtypes

In [ ]:
dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [ ]:
# Integer column analysis

In [ ]:
# replace ["ID", "y", "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"] with non integer columns
unique_values_dict = {}
for col in train_df.columns:
    if col not in ["ID", "y", "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
        unique_value = str(np.sort(train_df[col].unique()).tolist())
        tlist = unique_values_dict.get(unique_value, [])
        tlist.append(col)
        unique_values_dict[unique_value] = tlist[:]
for unique_val, columns in unique_values_dict.items():
    print("Columns containing the unique values : ",unique_val)
    print(columns)
    print("--------------------------------------------------")

In [ ]:
# For above -> consider excluding columns with only one value like 0.

In [ ]:
## Categorical column analysis

In [ ]:
# Analyse categorical column using stripplot

In [ ]:
var_name = ''
label_name = ''

col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.stripplot(x=var_name, y=label_name, data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
# Analyse categorical column using boxplot

In [ ]:
var_name = ''
label_name = ''

col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.boxplot(x=var_name, y=label_name, data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()


In [ ]:
# Analyse categorical column using violinplot

In [ ]:
var_name = ''
label_name = ''

col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.violinplot(x=var_name, y=label_name, data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
## Binary variable analysis [0,1]

In [ ]:
# lets start by getting the number of 0's and 1's in each of these variables

In [ ]:
zero_count_list = []
one_count_list = []
cols_list = unique_values_dict['[0, 1]']
for col in cols_list:
    zero_count_list.append((train_df[col]==0).sum())
    one_count_list.append((train_df[col]==1).sum())

N = len(cols_list)
ind = np.arange(N)
width = 0.35

plt.figure(figsize=(6,100))
p1 = plt.barh(ind, zero_count_list, width, color='red')
p2 = plt.barh(ind, one_count_list, width, left=zero_count_list, color="blue")
plt.yticks(ind, cols_list)
plt.legend((p1[0], p2[0]), ('Zero count', 'One Count'))
plt.show()

In [ ]:
# Now let us check the mean y value in each of the binary variable.
# Binary variables which shows a good color difference in the above graphs between 0 and 1 are 
# likely to be more predictive given the the count distribution is also good between both the classes 
# (can be seen from the previous graph)

In [ ]:
zero_mean_list = []
one_mean_list = []
cols_list = unique_values_dict['[0, 1]']
for col in cols_list:
    zero_mean_list.append(train_df.ix[train_df[col]==0].y.mean())
    one_mean_list.append(train_df.ix[train_df[col]==1].y.mean())

new_df = pd.DataFrame({"column_name":cols_list+cols_list, "value":[0]*len(cols_list) + [1]*len(cols_list), "y_mean":zero_mean_list+one_mean_list})
new_df = new_df.pivot('column_name', 'value', 'y_mean')

plt.figure(figsize=(8,80))
sns.heatmap(new_df)
plt.title("Mean of y value across binary variables", fontsize=15)
plt.show()

In [ ]:
## Important Variables

In [ ]:
# using xgboost model to get the important variables

In [ ]:
for f in ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values)) 
        train_df[f] = lbl.transform(list(train_df[f].values))        
#above is for categorical columns (objects, not 0/1)- basically to one hot code the feature

train_y = train_df['y'].values
train_X = train_df.drop(["ID", "y", "eval_set"], axis=1)

# Thanks to anokas for this #
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1
}
dtrain = xgb.DMatrix(train_X, train_y, feature_names=train_X.columns.values)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=100, feval=xgb_r2_score, maximize=True)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
# using random forest model to get the important variables

In [ ]:
from sklearn import ensemble
model = ensemble.RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)
model.fit(train_X, train_y)
feat_names = train_X.columns.values

## plot the importances ##
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1][:20]

plt.figure(figsize=(12,12))
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices], color="r", align="center")
plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical')
plt.xlim([-1, len(indices)])
plt.show()

In [ ]:
###RANDOM NOTES
# joining dfs
# train_df = pd.merge(train_df, prop_df, on='parcelid', how='left')